In [ ]:
## 利用ライブラリのインストール
!pip install --upgrade diffusers accelerate transformers
!pip install flask
!pip install pyngrok
!pip install python-socketio
!pip install line-bot-sdk
!pip install openai

In [ ]:
# 環境変数
SD_TOKEN="XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX"
# LINEのチャネルシークレット
LINE_CHANNEL_SECRET = 'XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX'
# LINEのチャネルアクセストークン
LINE_ACCESS_TOKEN = 'XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX'
# ngrokのAuthtoken
NGROK_AUTHTOKEN = 'XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX'

# OpenAI API-KEY
OPENAI_APIKEY =  'XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX'

# 必須フォルダ作成
!mkdir templates
!mkdir static


In [ ]:
IMAGEGEN_SYSTEM_PROMPT = """
あなたは優秀な画像プロンプトエンジニアです。

###指示
DALL·E APIへのPromptを出力形式に従って作成してください。生成画像の精度を上げるため、プロンプトは英語でお願いします。重要事項がある場合はその内容を最大限に考慮してください。

###出力形式
{
  "dalle-prompt": ""
}

###重要事項
ユーザーが入力した武勇伝を理解し、もっとも見せ場となるシーンの画像を生成するプロンプトを作成してください。

###観点
・画像のタイプを適切に選択（油絵、アニメ、カートゥーン）
・選択された画質に準拠する（例:高画質、4K、8K、HDR、低画質、8bit）
・主要な被写体を中心に配置（例: 人、動物、建物、風景、物体など）
・背景を考慮して配置（例: 山、海、都市、部屋、空など）
・アクションや状況を表現（例: 歩いている、飛んでいる、笑っている、眠っているなど）
・スタイルやムードを反映（例: 明るい、暗い、抽象的ななど）
・色の指示に従って色を適用（例: 青い、赤い、暖色系、冷色系など）
・追加の詳細やアクセサリーを追加（例: 帽子をかぶって、傘をさして、ギターを持っているなど）

###プロンプト文の補完項目
・画像のタイプ: [ファンタジーアートスタイル]
・画質: [高画質]
・主要な被写体: [選択した主要な被写体]
・背景: [選択した背景]
・アクションや状況: [選択したアクションや状況]
・スタイルやムード: [選択したスタイルやムード]
・色の指示: [選択した色]
・追加の詳細やアクセサリー: [選択した詳細やアクセサリー]

###画像生成の目的
ユーザーはポップな絵が好みです。武勇伝を象徴する画像を要求しています。
"""

In [ ]:
BUYUUDEN_SYSTEM_PROMPT = """」をから武勇伝を生成してください。
誇張した表現を多く盛り込んで、だれもが尊敬する武勇伝に仕上げてください。
生成の際、OpenAIの安全性に関するポリシーに抵触する表現は避けてください。
作成した武勇伝は出力形式に従って出力してください。

###出力形式
{
  "buyuden-text": ""
}

"""

In [ ]:
import openai
import requests
from PIL import Image
from io import BytesIO
from datetime import timedelta
import uuid
import hashlib
import json

def remove_control_characters(s):
    return ''.join(ch for ch in s if ord(ch) >= 32 or ch in '\n\r\t')


# 画像生成
def image_generator(prompt):
    # OpenAIのAPIキーを設定
    openai.api_key = OPENAI_APIKEY

    response = openai.Image.create(
        prompt=prompt,
        n=1,
        size="1024x1024"
    )
    image_url = response['data'][0]['url']
    return image_url

# 画像生成プロンプト生成
def generate_prompt(prompt):
    openai.api_key = OPENAI_APIKEY
    response = openai.ChatCompletion.create(
        model="gpt-4",
        max_tokens=4096,
        messages=[
            {"role": "system", "content": IMAGEGEN_SYSTEM_PROMPT},
            {"role": "user", "content":prompt}
        ]
    )
    response_str = remove_control_characters(response['choices'][0]['message']['content'])
    result_json = json.loads(response_str, strict=False)

    return result_json['dalle-prompt']

# 武勇伝生成
def generate_buyuuden(text, user_name=None):
    openai.api_key = OPENAI_APIKEY
    name_pronpt = ''
    if user_name is not None:
        name_pronpt = '私の名前は' + user_name + ""

    response = openai.ChatCompletion.create(
        model="gpt-4",
        max_tokens=4096,
        messages=[
            {"role": "system", "content": BUYUUDEN_SYSTEM_PROMPT},
            {"role": "user", "content": name_pronpt + "「" + text + BUYUUDEN_SYSTEM_PROMPT}
        ]
    )
    response_str = remove_control_characters(response['choices'][0]['message']['content'])
    result_json = json.loads(response_str, strict=False)

    return result_json['buyuden-text']


In [ ]:
from flask import Flask, render_template, request, abort
from markupsafe import Markup
from linebot import LineBotApi, WebhookHandler
from linebot.exceptions import InvalidSignatureError
from linebot.models import MessageEvent, TextMessage, ImageMessage
from linebot.models import TextSendMessage, ImageSendMessage


from pyngrok import ngrok
import socketio

from PIL import Image
import io

import os
import stat
import tempfile
from IPython.display import Image, display



def handle_line_message(text):
    print("入力テキスト->" + text)

    # メッセージから武勇伝を作成する
    buyuuden_text = generate_buyuuden(text)
    print("武勇伝->" + buyuuden_text)

    # 武勇伝の画像の生成プロンプトを生成する
    create_prompt = generate_prompt(buyuuden_text)
    print("画像生成プロンプト->" + create_prompt)

    # 武勇伝画像を生成する
    original_content_url = image_generator(create_prompt)
    print(original_content_url)

    display(Image(url=original_content_url))

if __name__ == '__main__':
    handle_line_message('犬に吠えられた')

In [ ]:
%%writefile templates/index.html
<!DOCTYPE html>
<html>
<head>
    <title>Chat Application</title>
    <script src="https://cdnjs.cloudflare.com/ajax/libs/socket.io/4.1.3/socket.io.js"></script>
    <style>
        #image-container {
            position: relative;
            width: 64vw;
            height: auto;
            overflow: hidden;
            margin: 2em auto;
            border: 5px solid #ddd;
            border-radius: 15px;
            box-shadow: 0 0 10px rgba(0, 0, 0, 0.3);
        }
        #image-container img {
            max-width: 100%;
            max-height: 100%;
            display: block;
            margin: 0 auto;
            transition: top 1s;
        }
        #previous-images {
            display: flex;
            flex-wrap: wrap;
            justify-content: center;
        }
        #previous-images img {
            width: 15%;
            margin: 5px;
            cursor: pointer;
        }
    </style>
</head>
<body>
    <h1>楽らく武勇伝　速報</h1>
    <div id="image-container"></div>

<script>
    var socket = io();

    function displayNewImage(imageData) {
        var imageContainer = document.getElementById("image-container");

        // 新しいdivを作成して、そこに画像とキャプションを追加する
        var newDivImage = document.createElement("div");

        // 新しい画像を作成
        var mainImage = document.createElement("img");
        mainImage.src = imageData.url;
        newDivImage.appendChild(mainImage);

        // 画像のキャプションを表示
        var captionElement = document.createElement("p");
        captionElement.innerText = imageData.caption;
        captionElement.style = "margin-top: 10px; text-align: center;"; // キャプションの上に余白を追加
        newDivImage.appendChild(captionElement);
        imageContainer.insertBefore(newDivImage, imageContainer.firstChild);

    }

    socket.on("image", function(imageData) {
        displayNewImage(imageData);
    });
</script>

</body>
</html>


In [ ]:
from flask import Flask, render_template, request, abort
from markupsafe import Markup
from linebot import LineBotApi, WebhookHandler
from linebot.exceptions import InvalidSignatureError
from linebot.models import MessageEvent, TextMessage, ImageMessage
from linebot.models import TextSendMessage, ImageSendMessage


from pyngrok import ngrok
import socketio

from PIL import Image
import io

import os
import stat
import tempfile

line_bot_api = LineBotApi(LINE_ACCESS_TOKEN)
handler = WebhookHandler(LINE_CHANNEL_SECRET)

app = Flask(__name__, static_folder='static')
app.config['SECRET_KEY'] = 'secret!'
app.config['STATIC_FOLDER'] = 'static'

sio = socketio.Server(async_mode='threading')
app.wsgi_app = socketio.WSGIApp(sio, app.wsgi_app)

static_folder = app.static_folder
temp_dir = os.path.join(static_folder, 'temp')  # 一時ファイル用のディレクトリを作成
os.makedirs(temp_dir, exist_ok=True)  # ディレクトリが存在しない場合は作成

app.static_url_path = '/static'

public_url = ''

def get_original_content_url(image_bytes):
    with tempfile.NamedTemporaryFile(dir=temp_dir, delete=False) as temp_file:
        temp_file.write(image_bytes)
        temp_file.flush()
        image_filename = os.path.basename(temp_file.name)
        original_content_url = f"{public_url}/static/temp/{image_filename}"
        print('画像URL->' + original_content_url)

        file_path = temp_file.name
        os.chmod(file_path, stat.S_IRUSR | stat.S_IWUSR | stat.S_IRGRP | stat.S_IROTH)

        return original_content_url

@app.route('/')
def index():
    return render_template('index.html')

@sio.on('message')
def handle_message(sid, message):
    print('Received message: ' + message)
    sio.emit('message', message)

@app.route('/post', methods=['POST'])
def sample_form_temp():
    signature = request.headers['X-Line-Signature']
    body = request.get_data(as_text=True)
    app.logger.info("Request body: " + body)
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        print("Invalid signature. Please check your channel access token/channel secret.")
        abort(400)

    return 'OK'

@handler.add(MessageEvent, message=TextMessage)
def handle_line_message(event):
    text = event.message.text
    print("入力テキスト->" + text)

    try:
        profile = line_bot_api.get_profile(event.source.user_id)

        # メッセージから武勇伝を作成する
        buyuuden_text = generate_buyuuden(text, user_name=profile.display_name)
        print("武勇伝->" + buyuuden_text)



        # 武勇伝の画像の生成プロンプトを生成する
        create_prompt = generate_prompt(buyuuden_text)
        print("画像生成プロンプト->" + create_prompt)

        # 武勇伝画像を生成する
        original_content_url = image_generator(create_prompt)

        sio.emit('message', buyuuden_text)
        sio.emit('image', {'url': original_content_url, 'caption': buyuuden_text})
        reply_token = event.reply_token
        send_image_reply(reply_token, original_content_url)

        buyuuden_text_message = TextSendMessage(text=buyuuden_text)
        line_bot_api.push_message(event.source.user_id, buyuuden_text_message)
    except Exception as e:
        print(f'handle_line_message Exception: {e}')
        buyuuden_text_message = TextSendMessage(text='あなたの武勇伝は安全性に関するポリシーに抵触する恐れがあるため、処理を中断しました。')
        line_bot_api.push_message(event.source.user_id, buyuuden_text_message)


@handler.add(MessageEvent, message=ImageMessage)
def handle_image_message(event):
    image_message = event.message
    response = line_bot_api.get_message_content(image_message.id)
    image_bytes = response.content
    sio.emit('image', image_bytes)  # 画像データをブロードキャスト
    reply_token = event.reply_token
    original_content_url = get_original_content_url(image_bytes)
    send_image_reply(reply_token, original_content_url)

def send_image_reply(reply_token, original_content_url):
    # 画像をバイト列からImageSendMessageに変換
    image_message = ImageSendMessage(
        original_content_url=original_content_url,
        preview_image_url=original_content_url
    )

    # LINEに画像を送信
    line_bot_api.reply_message(reply_token, image_message)

if __name__ == '__main__':
    ngrok.set_auth_token(NGROK_AUTHTOKEN)
    https_tunnel = ngrok.connect(addr='127.0.0.1:5000', bind_tls=True)
    print(https_tunnel)
    public_url = https_tunnel.public_url

    app.run()
